In [ ]:
# ! pip install duckduckgo-search beautifulsoup4 requests
# ! pip install langchain duckduckgo-search ollama


In [2]:
from duckduckgo_search import DDGS
import requests
from bs4 import BeautifulSoup
import subprocess

In [ ]:
# 1. Fazer a busca no DuckDuckGo
def buscar_links(query, max_results=5):
    with DDGS() as ddgs:
        resultados = ddgs.text(query)
        return [r["href"] for r in resultados][:max_results]

# 2. Extrair conteúdo da página
def extrair_texto(url):
    try:
        resp = requests.get(url, timeout=10)
        soup = BeautifulSoup(resp.content, "html.parser")
        return ' '.join(p.get_text() for p in soup.find_all('p'))
    except Exception as e:
        return f"[Erro ao acessar {url}: {e}]"

# 3. Enviar para o Ollama local com o modelo llama3
def perguntar_ollama(prompt, model="llama3.2"):
    process = subprocess.Popen(
        ["ollama", "run", model],
        stdin=subprocess.PIPE,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True
    )
    resposta, erro = process.communicate(input=prompt)
    return resposta.strip()

# 4. Pipeline completa
def agente_viagem(query):
    print(f"🔍 Buscando por: {query}")
    links = buscar_links(query)
    print(f"🔗 Links encontrados: {links}\n")

    textos = [extrair_texto(link) for link in links]
    texto_compilado = '\n\n'.join(textos[:3])  # limite para não sobrecarregar

    prompt = (
        f"Considere o texto a seguir com informações sobre viagens para '{query}':\n\n"
        f"{texto_compilado}\n\n"
        "Faça um resumo dos melhores destinos e compare preços, se possível."
    )

    resposta = perguntar_ollama(prompt)
    print(f"\n🧠 Resumo do modelo:\n{resposta}")


In [5]:
agente_viagem("melhores destinos para viajar saindo de Natal site: https://www.clubtravel.ie/")


🔍 Buscando por: melhores destinos para viajar saindo de Natal site: https://www.clubtravel.ie/
🔗 Links encontrados: []


🧠 Resumo do modelo:
Olá! Vou fazer o resumo dos melhores destinos para viajar saindo de Natal e comparar preços com base no site Club Travel.

**Melhores Destinos:**

1. **Alicante (Espanha)**:
 * Tempo de voo: 3h25m
 * Clima: Quente e soleiro
 * Atrações: Praias, cidade histórica, montanhas
2. **Nice (França)**:
 * Tempo de voo: 2h30m
 * Clima: Misto (verão quente, inverno frio)
 * Atrações: Praia de Nice, Catedral de São Renato, Mercado de Ópera
3. **Amsterdã (Holanda)**:
 * Tempo de voo: 1h45m
 * Clima: Misto (verão quente, inverno frio)
 * Atrações: Canais, Rijksmuseum, Varsaviana
4. **Lisboa (Portugal)**:
 * Tempo de voo: 2h50m
 * Clima: Quente e soleiro
 * Atrações: Praia da Bela Vista, Castelo de São Jorge, Mosteiro dos Jerónimos
5. **Barcelona (Espanha)**:
 * Tempo de voo: 2h30m
 * Clima: Quente e soleiro
 * Atrações: La Rambla, Barceloneta, Sagrada Família

